# MBA DSA USP/ESALQ 2024.1 Time 2
## Desafio House Prices

A ser feito ainda:
- Melhor imputação dos NaN
- Verificar limpeza dos dados
- Ver sobre feature engeneering
- Seleção de variáveis?
- Testes estatísticos

### Limpeza e ajuste dos dados

Importação das bibliotecas necessárias

In [1]:
import pandas as pd

Leitura dos dados do arquivo para treino ("train.csv") e armazenando no objeto 'dados_brutos'

In [2]:
dados_brutos = pd.read_csv("train.csv")

print("Quantidade de linhas e colunas")
print(dados_brutos.shape)

print("\nTipos de objetos do DataFrame")
print(dados_brutos.dtypes)

#print("\nInformações sobre o banco de dados (coluna, contador de nulos, e tipo de informação)")
#dados_brutos.info()

Quantidade de linhas e colunas
(1460, 81)

Tipos de objetos do DataFrame
Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object


O banco de dados possui uma série de colunas (variáveis) que possuem valores nulos, algumas em maior quantidade e outras em menor quantidade:

In [3]:
nulos = dados_brutos.isnull().sum()
nulos_ordenados = nulos.sort_values(ascending=False)

print("Quantidade de valores nulos por coluna:")
print(nulos_ordenados.head(20))
print("Como o último valor já é zero, depois desse todos são zerados mesmo")


Quantidade de valores nulos por coluna:
PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
Id                 0
dtype: int64
Como o último valor já é zero, depois desse todos são zerados mesmo


Como são valores absolutos, é mais fácil excluir do banco de dados as variáveis que possuam valores nulos acima de um determinado percentual do total (eu usei 15% como delimitador), salvando no objeto 'dados_filtrados'

In [4]:
nulos_percentuais = nulos/len(dados_brutos.index)
var_excluir = dados_brutos.columns[nulos_percentuais > 0.15]
dados_filtrados = dados_brutos.drop(var_excluir, axis = 1)

print("Quantidade de linhas e colunas depois de ajustado")
print(dados_filtrados.shape)

print("\nTipos de objetos do DataFrame")
print(dados_filtrados.dtypes)

#print("\nInformações sobre o banco de dados (coluna, contador de nulos, e tipo de informação)")
#dados_filtrados.info()

Quantidade de linhas e colunas depois de ajustado
(1460, 75)

Tipos de objetos do DataFrame
Id                int64
MSSubClass        int64
MSZoning         object
LotArea           int64
Street           object
                  ...  
MoSold            int64
YrSold            int64
SaleType         object
SaleCondition    object
SalePrice         int64
Length: 75, dtype: object


Isso feito, existem variáveis que são 'object' (não-numérico, então qualitativas ou categóricas) e variáveis que são "intXX" (portanto, numéricas). Por questão de facilidade e considerando o tempo disponível, por enquanto vou desconsiderar as variáveis 'quali' e vou utilizar apenas as 'quanti'.

In [5]:
colunas_quali = dados_filtrados.columns[dados_filtrados.dtypes == "object"]
variaveis_quanti = dados_filtrados.drop(colunas_quali, axis = 1)
#print("AS VARIÁVEIS NUMÉRICAS ('QUANTI') ENTÃO SÃO 37:")
#print(variaveis_quanti.columns)
#print("\n")

colunas_quanti = dados_filtrados.columns[dados_filtrados.dtypes != "object"]
variaveis_quali = dados_filtrados.drop(colunas_quanti, axis = 1)
#print("AS VARIÁVEIS NÃO-NÚMERICAS ('QUALI') FICAM SENDO APENAS 38:")
#print(variaveis_quali.columns)

 Filtro das variáveis quali e quanti para evitar confusão. Contudo, ainda existem valores nulos no meio dos dados (foram eliminados só as variações com nulos >= 15% do total). Necessário realizar a imputação esses valores, pois nenhuma análise é feita com qualquer valor nulo: quebra o sistema.

 Link sobre o carregamento das bibliotecas para imputar dados: https://medium.com/@sanjushusanth/missing-value-imputation-techniques-in-python-62aeab65a6a6

 Por mera facilidade momentânea, serão utilizados os dados de média e mediana para as imputações, ou, ainda seguindo o que alguns consideram 'boas práticas', simplesmente registrando um '-1' no valor imputado. Fiz um banco de dados para cada tipo dessas imputações a fim de testar qual melhor se encaixa nessa simples imputação (sem cálculos próprios de imputação para esse fim).

In [6]:
variaveis_quanti_namum = variaveis_quanti.fillna(-1)
variaveis_quanti_namediana = variaveis_quanti.fillna(variaveis_quanti.median())
variaveis_quanti_namedia = variaveis_quanti.fillna(variaveis_quanti.mean())

print("VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO '-1' AINDA COM VALORES NULOS...")
print(variaveis_quanti_namum.columns[variaveis_quanti_namum.isnull().sum() > 0])

print("\n VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MÉDIA' AINDA COM VALORES NULOS...")
print(variaveis_quanti_namum.columns[variaveis_quanti_namedia.isnull().sum() > 0])

print("\n VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MEDIANA' AINDA COM VALORES NULOS...")
print(variaveis_quanti_namum.columns[variaveis_quanti_namediana.isnull().sum() > 0])

try:
    variaveis_quanti_namum.columns[variaveis_quanti_namediana.isnull().sum() > 0] + variaveis_quanti_namum.columns[variaveis_quanti_namedia.isnull().sum() > 0]+ variaveis_quanti_namum.columns[variaveis_quanti_namum.isnull().sum() > 0]
    print("\nTUDO CERTO!")
except:
    print("\n!!! DEU ALGUM ERRO !!!")

VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO '-1' AINDA COM VALORES NULOS...
Index([], dtype='object')

 VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MÉDIA' AINDA COM VALORES NULOS...
Index([], dtype='object')

 VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MEDIANA' AINDA COM VALORES NULOS...
Index([], dtype='object')

TUDO CERTO!


Definição da variável dependente (y) e das variáveis independentes (x) de cada um dos bancos de dados anteriores:

In [7]:
x_namum = variaveis_quanti_namum.drop("SalePrice", axis = 1)
x_namedia = variaveis_quanti_namedia.drop("SalePrice", axis = 1)
x_namediana = variaveis_quanti_namediana.drop("SalePrice", axis = 1)
y = variaveis_quanti.SalePrice # como o 'y' é igual para todos (o preço de venda), então tanto faz a base de dados escolhida

### Separação dos dados para treino e teste

Importação das bibliotecas adicionais necessárias

In [8]:
from sklearn.model_selection import train_test_split

Separação do banco de dados entre treino (0,80 ou 80%) e teste (0,20 ou 20%) - proporções arbitrárias, eu que defini assim. Pode-se testar com proporções diferentes. Usa-se o scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Para garantir consistência entre os resultados, utilizado sempre _random_state = 1_

In [9]:
# Separação treino e teste dos dados imputados com -1
treino_namum, teste_namum, treino_y_um, teste_y_um = train_test_split(x_namum, y, test_size = 0.2, random_state = 1)

# Separação treino e teste dos dados imputados com a média
treino_namedia, teste_namedia, treino_y_media, teste_y_media = train_test_split(x_namedia, y, test_size = 0.2, random_state = 1)

# Separação treino e teste dos dados imputados com a mediana
treino_namediana, teste_namediana, treino_y_mediana, teste_y_mediana = train_test_split(x_namediana, y, test_size = 0.2, random_state = 1)

### Modelagem e posterior avaliação dos resultados
Com os dados já limpos, já separados pelas imputações diferentes para verificação e com as bases para treino e teste, começa-se efetivamente a modelagem através de alguns algoritmos diferentes a fim de testar precisão dos resultados estimados.

Importação das bibliotecas adicionais necessárias

In [26]:
from sklearn import tree # biblioteca para modelagem através de decision tree (árvore de decisão)
from sklearn.neighbors import KNeighborsRegressor # biblioteca para modelagem através de KNN
from sklearn.ensemble import GradientBoostingRegressor # biblioteca para modelagem com Gradient Boost
from sklearn.linear_model import LinearRegression # biblioteca para modelagem através de regressão Linear Simples
from sklearn.linear_model import LogisticRegression # biblioteca para modelagem com Logistic Regression
from sklearn.linear_model import Ridge # biblioteca para modelagem com Ridge
from sklearn import linear_model # biblioteca para modelagem com Bayesian Ridge

#### Documentação
Linear Regression (Regressão Linear)
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

Decision Tree (Árvore de Decisão)
https://scikit-learn.org/stable/modules/tree.html#regression

KNN - K Nearest Neighbors (Vizinhança Próxima)
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

Gradient Boost Regressor
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor

Logistic Regression (Modelo Logístico, 'Logit' ou Classificador de Máxima Entropia)
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

Ridge
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge

Bayesian Ridge Regression
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html#sklearn.linear_model.BayesianRidge

#### NaN imputado com -1

In [32]:
# Regressão Linear Simples (Linear Regression)
namum_rl = LinearRegression().fit(treino_namum, treino_y_um)
y_namum_rl = namum_rl.predict(teste_namum)

# Árvore de Decisão (decision tree)
namum_tree = tree.DecisionTreeRegressor(random_state = 1).fit(treino_namum, treino_y_um)
y_namum_tree = namum_tree.predict(teste_namum)

#(K Vizinhos Mais Próximos - KNN (K Nearest Neighbors))
namum_knn = KNeighborsRegressor(n_neighbors = 2, weights = "uniform").fit(treino_namum, treino_y_um)
y_namum_knn = namum_knn.predict(teste_namum)

#Gradient Boost Regressor
namum_gbr = GradientBoostingRegressor(random_state = 1).fit(treino_namum, treino_y_um)
y_namum_gbr = namum_gbr.predict(teste_namum)

#Logistic Regression
namum_lor = LogisticRegression(random_state = 1, max_iter = 1000).fit(treino_namum, treino_y_um)
y_namum_lor = namum_lor.predict(teste_namum)

#Ridge
namum_ridge = Ridge(alpha = 1.0).fit(treino_namum, treino_y_um)
y_namum_ridge = namum_ridge.predict(teste_namum)

#Bayesian Ridge Regression
namum_brr = linear_model.BayesianRidge().fit(treino_namum, treino_y_um)
y_namum_brr = namum_brr.predict(teste_namum)

/usr/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### NaN imputado com Média

In [33]:
# Regressão Linear Simples (Linear Regression)
namedia_rl = LinearRegression().fit(treino_namedia, treino_y_media)
y_namedia_rl = namedia_rl.predict(teste_namedia)

# Árvore de Decisão (decision tree)
namedia_tree = tree.DecisionTreeRegressor(random_state = 1).fit(treino_namedia, treino_y_media)
y_namedia_tree = namedia_tree.predict(teste_namedia)

#(K Vizinhos Mais Próximos - KNN (K Nearest Neighbors))
namedia_knn = KNeighborsRegressor(n_neighbors = 2, weights = "uniform").fit(treino_namedia, treino_y_media)
y_namedia_knn = namedia_knn.predict(teste_namedia)

#Gradient Boost Regressor
namedia_gbr = GradientBoostingRegressor(random_state = 1).fit(treino_namedia, treino_y_media)
y_namedia_gbr = namedia_gbr.predict(teste_namedia)

#Logistic Regression
namedia_lor = LogisticRegression(random_state = 1, max_iter = 1000).fit(treino_namedia, treino_y_media)
y_namedia_lor = namedia_lor.predict(teste_namedia)

#Ridge
namedia_ridge = Ridge(alpha = 1.0).fit(treino_namedia, treino_y_media)
y_namedia_ridge = namedia_ridge.predict(teste_namedia)

#Bayesian Ridge Regression
namedia_brr = linear_model.BayesianRidge().fit(treino_namedia, treino_y_media)
y_namedia_brr = namedia_brr.predict(teste_namedia)

/usr/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### NaN imputado com Mediana

In [34]:
# Regressão Linear Simples (Linear Regression)
namediana_rl = LinearRegression().fit(treino_namediana, treino_y_mediana)
y_namediana_rl = namediana_rl.predict(teste_namediana)

# Árvore de Decisão (decision tree)
namediana_tree = tree.DecisionTreeRegressor(random_state = 1).fit(treino_namediana, treino_y_mediana)
y_namediana_tree = namediana_tree.predict(teste_namediana)

#(K Vizinhos Mais Próximos - KNN (K Nearest Neighbors))
namediana_knn = KNeighborsRegressor(n_neighbors = 2, weights = "uniform").fit(treino_namediana, treino_y_mediana)
y_namediana_knn = namediana_knn.predict(teste_namediana)

#Gradient Boost Regressor
namediana_gbr = GradientBoostingRegressor(random_state = 1).fit(treino_namediana, treino_y_mediana)
y_namediana_gbr = namediana_gbr.predict(teste_namediana)

#Logistic Regression
namediana_lor = LogisticRegression(random_state = 1, max_iter = 1000).fit(treino_namediana, treino_y_mediana)
y_namediana_lor = namediana_lor.predict(teste_namediana)

#Ridge
namediana_ridge = Ridge(alpha = 1.0).fit(treino_namediana, treino_y_mediana)
y_namediana_ridge = namediana_ridge.predict(teste_namediana)

#Bayesian Ridge Regression
namediana_brr = linear_model.BayesianRidge().fit(treino_namediana, treino_y_mediana)
y_namediana_brr = namediana_brr.predict(teste_namediana)

/usr/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Avaliação dos resultados realizada através de RMSE (root mean squared error), no caso deste desafio.

Importação das bibliotecas adicionais necessárias

In [35]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error
import math

#### Análises (pela imputação)

##### NaN por -1

In [36]:
print("Linear Regression")
print(root_mean_squared_error(teste_y_um, y_namum_rl))
print("\nDecision Tree")
print(root_mean_squared_error(teste_y_um, y_namum_tree))
print("\nKNN")
print(root_mean_squared_error(teste_y_um, y_namum_knn))
print("\nGBR")
print(root_mean_squared_error(teste_y_um, y_namum_gbr))
print("\nLogistic Regression")
print(root_mean_squared_error(teste_y_um, y_namum_lor))
print("\nRidge")
print(root_mean_squared_error(teste_y_um, y_namum_ridge))
print("\nBayesian Ridge")
print(root_mean_squared_error(teste_y_um, y_namum_brr))

Linear Regression
35621.27326115689

Decision Tree
35116.94890306209

KNN
53538.68340927549

GBR
22404.11312398332

Logistic Regression
56992.054928675374

Ridge
35608.081379361436

Bayesian Ridge
40229.11392691891


##### NaN por Média

In [37]:
print("Linear Regression")
print(root_mean_squared_error(teste_y_media, y_namedia_rl))
print("\nDecision Tree")
print(root_mean_squared_error(teste_y_media, y_namedia_tree))
print("\nKNN")
print(root_mean_squared_error(teste_y_media, y_namedia_knn))
print("\nGBR")
print(root_mean_squared_error(teste_y_media, y_namedia_gbr))
print("\nLogistic Regression")
print(root_mean_squared_error(teste_y_media, y_namedia_lor))
print("\nRidge")
print(root_mean_squared_error(teste_y_media, y_namedia_ridge))
print("\nBayesian Ridge")
print(root_mean_squared_error(teste_y_media, y_namedia_brr))

Linear Regression
36078.05339518298

Decision Tree
47410.39995583883

KNN
53401.849345830655

GBR
23146.583584564967

Logistic Regression
53848.77888867462

Ridge
36065.19504911939

Bayesian Ridge
40239.30702099713


##### NaN por Mediana

In [39]:
print("Linear Regression")
print(root_mean_squared_error(teste_y_mediana, y_namediana_rl))
print("\nDecision Tree")
print(root_mean_squared_error(teste_y_mediana, y_namediana_tree))
print("\nKNN")
print(root_mean_squared_error(teste_y_mediana, y_namediana_knn))
print("\nGBR")
print(root_mean_squared_error(teste_y_mediana, y_namediana_gbr))
print("\nLogistic Regression")
print(root_mean_squared_error(teste_y_mediana, y_namediana_lor))
print("\nRidge")
print(root_mean_squared_error(teste_y_mediana, y_namediana_ridge))
print("\nBayesian Ridge")
print(root_mean_squared_error(teste_y_mediana, y_namediana_brr))

Linear Regression
36055.310690629514

Decision Tree
34013.71426658524

KNN
53488.18385333421

GBR
23125.9818814507

Logistic Regression
51854.51240034369

Ridge
36042.55239994742

Bayesian Ridge
40228.513586626104


### O modelo para publicação

In [47]:
# GBR com mediana (23125.98188)

# Carregamento do dataset
teste = pd.read_csv("test.csv")

# Filtro das colunas utilizadas e imputação dos NaN
colunas_quali_teste = teste.columns[teste.dtypes == "object"]
variaveis_quanti_teste = teste.drop(colunas_quali_teste, axis = 1)
variaveis_quanti_teste = variaveis_quanti_teste.drop("LotFrontage", axis = 1)
variaveis_quanti_teste = variaveis_quanti_teste.fillna(variaveis_quanti_teste.median())

# Gerando as previsões
y_gbr = namediana_gbr.predict(variaveis_quanti_teste)

### Gerar arquivo de submissão
Pontuação 0.14569 (quanto menor, melhor)

In [52]:
teste["SalePrice"] = y_gbr
resultado = teste[["Id", "SalePrice"]]
resultado.to_csv("resultado.csv", index=False)